In [1]:
pip install Mosek

     --------------------------------------- 19.5/19.5 MB 13.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\abhis\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [1]:
import mosek
import sympy as sp
import cvxpy as cp

In [2]:
# Define the symbolic variables
x = sp.Symbol('x')
x_t = sp.Symbol('x_t')
x_t_next = sp.Symbol('x_(t+1)')

# Define the system equation
system_equation = sp.Eq(x_t_next, x_t *(1/2))

#defining constraints
g1 = 100-x**2 >= 0

# Print the system equation
print("System equation:")
sp.pprint(system_equation, use_unicode=True)
print("Constraints")
sp.pprint(g1, use_unicode=True)

System equation:
x₍ₜ₊₁₎ = 0.5⋅xₜ
Constraints
       2    
100 - x  ≥ 0


In [4]:
a = sp.Symbol('a')
b = sp.Symbol('b')
c = sp.Symbol('c')


epsilon = sp.Symbol('epsilon')

# Define the ranking function V(x)
V = a * x**2 + b * x + c

# Substitute x_t with x_t / 2 in the ranking function to reflect the state transition
V1 = V.subs(x, x / 2)


inequality = sp.Le(V1, V - epsilon)

sp.pprint(inequality, use_unicode=True)

rearranged = sp.simplify(-V1 + V - epsilon)
print("rearranged")
sp.pprint(rearranged, use_unicode=True)

to_solve = rearranged >= 0


   2                               
a⋅x    b⋅x          2              
──── + ─── + c ≤ a⋅x  + b⋅x + c - ε
 4      2                          
rearranged
     2          
3⋅a⋅x    b⋅x    
────── + ─── - ε
  4       2     


In [5]:
to_solve

3*a*x**2/4 + b*x/2 - epsilon >= 0

In [6]:
coeffs = sp.Poly(rearranged, x).all_coeffs()

In [8]:
print("Coefficients of the polynomial:")
for i, coeff in enumerate(coeffs):
    print(f"x^{len(coeffs)-1-i}: {coeff}")

Coefficients of the polynomial:
x^2: 3*a/4
x^1: b/2
x^0: -epsilon


In [9]:
n = len(coeffs) - 1
A = cp.Variable((n+1, n+1), symmetric=True)

In [10]:
A

Variable((3, 3), var1, symmetric=True)

In [11]:
constraints = [
    cp.trace(A) == 1,
]


In [12]:
constraints

[Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, NONNEGATIVE, ()))]

In [13]:
poly_expr = sum(coeffs[i] * cp.power(x, i) for i in range(len(coeffs)))

TypeError: Cannot convert expression to float